# Experiments

We're considering performace of an SVM classifier (OVR scheme) for 4 classes, using three different kernel functions. To train and test the SVM classifier we'll have to provide it with the similarity matrices.

Training - requires NxN Gram matrix (square matrix compromised of values of the kernel function between pairs training examples)

Testing - requires MxN matrix (element i,j is the value of the kernel function between i-th example of the testing set and j-th example of the training set)

N = cardinaliry of the training set, M = cardinality of the testing set

In [2]:
import pandas as pd
import numpy as np

In [3]:
def similarity_matrix(first_dataset, second_dataset, similarity_function):
    """
    Calculate the similarity matrix between elements of two datasets using a similarity function.

    Args:
    - first_dataset: List or array-like, the first dataset
    - second_dataset: List or array-like, the second dataset
    - similarity_function: Function, the similarity function that takes two elements as arguments

    Returns:
    - similarity_matrix: NumPy ndarray, the similarity matrix
    """

    # Ensure that the datasets are iterable
    first_dataset = np.array(first_dataset)
    second_dataset = np.array(second_dataset)

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((len(first_dataset), len(second_dataset)))

    # Populate the similarity matrix
    for i in range(len(first_dataset)):
        for j in range(len(second_dataset)):
            similarity_matrix[i, j] = similarity_function(first_dataset[i], second_dataset[j])

    return similarity_matrix

## Kernel functions

We're considering SSK, NGK and WK kernel functions, each with its own set of hiperparameters. 

SSK is parameterised by $k$ = length of the substrings used for feature mapping/kernel computation and $\lambda$= real numer from the interval [0,1] which indicates how much we penalise the noncontiguity of the appeared substring in the imput document. 

NGK is parameterised with $n$, corresponding to $k$ in SSk.

WK has no hiperparameters

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support


def WK_SWM(X_train, y_train, X_test, y_test):  #a multi-class classifier

    '''calculates f1, precision, and recall for a SVM classifer using linear tfidf mapping
    Args:
    - X_train,y_train, X_test, y_test
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel, Class, F1, Precision , Recall
    '''
    vectorizer = TfidfVectorizer(sublinear_tf=True, use_idf=True, smooth_idf=True, norm='l2',
                                 analyzer='word', stop_words='english')

    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(X_train_tfidf, y_train)

    y_pred = svm_classifier.predict(X_test_tfidf)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    results_df = pd.DataFrame({
    'Kernel' : 'WK',
    'Class': range(len(precision)),
    'F1-Score': f1,
    'Precision': precision,
    'Recall': recall,
    })
    return results_df


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

def NGK_SVM(X_train, y_train, X_test, y_test, k):
    '''calculates f1, precision, and recall for a SVM classifer using linear n-gram mapping
    Args:
    - X_train,y_train, X_test, y_test, k=lenthg of the ngrams
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel,k, Class, F1, Precision , Recall
    '''
    ngram_range = (k, k)
    vectorizer = CountVectorizer(analyzer='char', ngram_range=ngram_range)

    x_train_ngrams = normalize(vectorizer.fit_transform(X_train), norm='l2')  #ngram vectors normalised to l2 norm
    x_test_ngrams = normalize(vectorizer.transform(X_test), norm='l2')

    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(x_train_ngrams, y_train)

    y_pred = svm_classifier.predict(x_test_ngrams)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

    results_df = pd.DataFrame({
        'Kernel':'NGK',
        'k':k,
        'Class': range(len(precision)),
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall
        })
    return results_df